# Práctica 2

Este notebook esta dedicado a resolver los ejercicios de la práctica 2 del Diplomado de Ciencia de Datos en la Facultad de Estudios Superiores Acatlán.

## Paqueterías

In [1]:
#import nltk
#nltk.download('stopwords')
#!pip install textblob

In [2]:
#!pip install cufflinks
#import cufflinks as cf
#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)

In [3]:
import ibm_boto3
from botocore.client import Config
import pandas as pd
from datetime import datetime
import re
#from unidecode import unidecode
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import numpy as np

## Datos

1. Exporta una conversación entre dos personas de Whatsapp, la forma de exportarla como txt es la siguiente: Primero seleccionas la opción "Más" sobre tu conversación , posteriormente seleccionas la opción de "Exportar chat" y finalmente eliges la opción "SIN ARCHIVOS", de esta forma se exporta la conversación en .txt a algún correo.

<font color='#800000'> La conversación fue bajada y depositada en un Cloud Object Storage, las siguientes tres celdas estan dedicadas a obtener los datos y no puden ser accesados sin las credenciales, por lo cual se puede omitir estas celdas. </font>

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_cos["IBM_API_KEY_ID"] ,
    ibm_auth_endpoint=credentials_cos["IBM_AUTH_ENDPOINT"],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_cos["ENDPOINT"])

In [6]:
cos.download_file('diplomadocienciadedatos-donotdelete-pr-ni0xvyxds7qosj', 'WhatsApp Chat with Isaac Carrada, Elbuenizzy.txt', 'conversacion_con_Elbuenizzy.txt')

2. El documento con extensión .txt transformar en un DataFrame de la siguiente forma:

In [7]:
def read_whatsapp_conversation(data):
    '''
    Read the data of a Whatsapp conversation where the information was download in a txt file and tranform the data to a DataFrame.
    
    Parameters:
    
    data: str, Route where is the file
    
    Author: Adolfo Cruz
    Contact: adolfo.ollin@gmail.com
    
    '''
    # Read data  
    aux = open('conversacion_con_Elbuenizzy.txt')
    text_all_conversations = aux.read().replace("\n", " ")

    dates_and_messagues_list = re.split(pattern='(\d+/\d+/\d+, \d+:\d+ )', string=text_all_conversations)[1:]

    messagues_list = []
    for k in range(0,len(dates_and_messagues_list),2):
        messagues_list.append(dates_and_messagues_list[k] + dates_and_messagues_list[k+1])
    
    # Defining the columns
    d_fecha = []
    v_hora = []
    t_texto = []
    v_usuario = []
    
    errs = []
    
    def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
        cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
        cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
        cleaned_text = u' '.join(cleaned_text.lower().strip().split())
        return cleaned_text
    
    for line in messagues_list[1:]:
        try:
            aux_date, aux_text = line.split('-',1)
            date = datetime.strptime(aux_date, '%m/%d/%y, %I:%M %p ')
            
            name, text = aux_text.split(':',1)
            name = clean_text(name)
            text = clean_text(text)
            

            d_fecha.append(date)
            v_hora.append(date.hour)
            t_texto.append(text)
            v_usuario.append(name)
        
        except ValueError:
            #print(line)
            #We can review the particular errors to see the way how we can handle
            errs.append(line)
        
            
            

    dic_aux = {'d_fecha': d_fecha, 'v_hora': v_hora, 't_texto':t_texto, 'v_usuario':v_usuario}
    
    return pd.DataFrame(dic_aux), errs
        
        
    
    
    
    

In [8]:
df, errs = read_whatsapp_conversation('conversacion_con_Elbuenizzy.txt')

In [9]:
df.head()

,d_fecha,v_hora,t_texto,v_usuario
0,2019-12-07 07:18:00,7,kha,isaac carrada elbuenizzy
1,2019-12-09 16:32:00,16,coo se apellida luis felipe,isaac carrada elbuenizzy
2,2019-12-10 17:44:00,17,sabes que acepta speech to text de extension,isaac carrada elbuenizzy
3,2019-12-12 21:04:00,21,chi,adolfo cruz
4,2019-12-12 21:04:00,21,es wav,adolfo cruz


In [10]:
errs

[]

3. Una vez teniendo la tabla en esa estructura procedemos a hacer lo siguiente:
  - (a) Haz el conteo de missings por variable y crea un dataframe con el siguiente formato y guardarlo en una variable llamada "missings" - En caso de tener ”missings” eliminar los registros de tu DataFrame

In [11]:
missings = pd.DataFrame(df.isnull().sum())
missings.reset_index(inplace=True)
missings=missings.rename(columns={"index":"nombre_columna",0:"numero_missings"})

In [12]:
missings

,nombre_columna,numero_missings
0,d_fecha,0
1,v_hora,0
2,t_texto,0
3,v_usuario,0


  - (b) Posteriormente haz el conteo de registros que tienen como valor "Multimedia omitido" en la variable "t_texto" y asignala a una variable llamada "total_multimedia", adicional a lo anterior elimina dichos registros. El número de registros mínimos para poder trabajar los siguientes pasos debería ser 200.

In [13]:
total_multimedia =  len(df.t_texto[df.t_texto == 'media omitted'])
print('Número de valores media omitted', total_multimedia)
df = df[df.t_texto != 'media omitted']

Número de valores media omitted 1150


  - (c) Verifica la calidad de los datos

<font color='#800000'> Este inciso era como encabezado, sin embargo me di cuenta de que hay muchos textos vacíos. Esto es porque algunos mensajes son emojies o solamente puntos como para recordar un mensaje pasado y a la hora de limpiar el texto como son carácteres especiales se quitan dejando la cadena vacía, y es por eso que los elimino en está sección. <font color='#800000'>

In [14]:
df.t_texto.value_counts()[:3]

          1236
jajaja     932
xd         404
Name: t_texto, dtype: int64

In [15]:
df = df[df.t_texto != '']

  - (d) Realiza la limpieza de la variable "v_usuario" , solo debes tener dos categorías en esta columna, el texto no debe contener caracteres especiales , en minúsculas y sin espacios.

In [16]:
df.v_usuario.value_counts(1)

isaac carrada elbuenizzy    0.56857
adolfo cruz                 0.43143
Name: v_usuario, dtype: float64

  - (e) La variable "d_fecha" conviertela a tipo "datetime"

In [17]:
# Este paso se realiza cuando se ejecuta la función read_whatsapp_conversation

  - (f) La variable "v_hora" conviertela a tipo entero

In [18]:
# Este paso se realiza dentro la función read_whatsapp_conversation
df.dtypes

d_fecha      datetime64[ns]
v_hora                int64
t_texto              object
v_usuario            object
dtype: object

  - (g) Realiza la limpieza de la variable "t_texto"

In [19]:
# Este paso se realiza cuando se ejecuta la función read_whatsapp_conversation

  - (h) Elimina stop words

In [20]:
stop_words = stopwords.words('spanish')

In [21]:
df.t_texto = df.t_texto.apply(lambda text: " ".join([x for x in text.split(" ") if x not in stop_words]))

  - (i) Elimina hapaxes

In [22]:
corpus=" ".join(df.t_texto.values)
fdist=nltk.FreqDist(corpus.split())
list_hapaxes=fdist.hapaxes()
print('Número de palabrás que aparecen solo una vez: ', len(list_hapaxes))
df.t_texto=df.t_texto.map(lambda text:" ".join([x for x in text.split(" ") if x not in list_hapaxes]))
df = df[df.t_texto != '']
print('Palabras eliminadas')

Número de palabrás que aparecen solo una vez:  8317
Palabras eliminadas


  - (j) Realiza tokenización

In [23]:
df.t_texto=df.t_texto.map(lambda x:x.split())

  - (k) Realiza derivación (Stemmer) en español, se puede utilizar de la siguiente forma:

In [24]:
spanish_stemmer = SnowballStemmer('spanish')
df.t_texto=df.t_texto.map(lambda text_list:[spanish_stemmer.stem(word) for word in text_list])

  - (l) Creación de variables(Se deben crear al menos 10 nuevas variables):

In [25]:
df['c_numero_palabras'] = df.t_texto.apply(lambda x: len(x))
df['c_numero_letras'] =df.t_texto.map(lambda x: sum(len(word) for word in x))
df['c_long_pro_pala'] = df['c_numero_letras'] / df['c_numero_palabras']

l = []
for element in df.t_texto:
    l = element + l
nuevas_variables = pd.Series(l).value_counts()[:40].index.tolist()
for col in nuevas_variables:
    df['v_'+col] = df.t_texto.apply(lambda lista_texto: int(col in lista_texto))

In [26]:
df["t_texto_unido"]=df.t_texto.map(lambda x:" ".join(x))
X_train = df.sample(frac=.5)


In [27]:
vect = TfidfVectorizer()
vect.fit(X_train["t_texto_unido"])
array_all= vect.transform(df["t_texto_unido"])
aux=pd.DataFrame(array_all.toarray(),columns=vect.get_feature_names())
df=pd.concat([df.reset_index(drop=True),aux.reset_index(drop=True)],axis=1)

In [28]:
df_text = df.copy()

In [29]:
df_text.head(2)

,d_fecha,v_hora,t_texto,v_usuario,c_numero_palabras,c_numero_letras,c_long_pro_pala,v_si,v_we,v_jajaj,...,yq,yug,yum,yuval,zanahori,zd,zenec,zon,zoom,zorrit
0,2019-12-07 07:18:00,7,[kha],isaac carrada elbuenizzy,1,3,3.000000,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-12-09 16:32:00,16,"[apell, luis, felip]",isaac carrada elbuenizzy,3,14,4.666667,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## PARTE 2 – Ingeniería Continuas

### Datos

In [30]:
# The code was removed by Watson Studio for sharing.

,Open,High,Low,Close,Adj Close,Volume,date
0,38.290001,40.160000,38.099998,39.880001,38.219864,3063098,2004-12-01 00:00:00
1,39.880001,40.139999,38.590000,40.000000,38.334866,469910,2004-12-02 00:13:00
2,39.799999,41.959999,39.520000,40.790001,39.091980,6223049,2004-12-03 00:38:00


In [31]:
df_bolsa = pd.read_csv('BOLSAA.MX.csv')

(a) Etiquetado de las variables

In [32]:
df_bolsa.rename(columns={'Open':'c_Open', 'High':'c_High', 'Low': 'c_Low', 'Close':'c_Close', 'Adj Close':'c_Adj_Close', 'Volume':'c_Volume', 'date':'d_date'}, inplace=True)

(b) Calidad de datos, para verificar que no existan anomalías

In [33]:
df_bolsa.isnull().sum()

c_Open         0
c_High         0
c_Low          0
c_Close        0
c_Adj_Close    0
c_Volume       0
d_date         0
dtype: int64

In [34]:
df_bolsa.describe()

,c_Open,c_High,c_Low,c_Close,c_Adj_Close,c_Volume
count,5040.000000,5040.000000,5040.000000,5040.000000,5040.000000,5.040000e+03
mean,43.288968,44.206151,42.444563,43.264167,42.340073,1.121456e+06
std,3.359412,3.200158,3.575667,3.371480,3.888366,9.929247e+05
min,29.100000,34.130001,28.270000,28.690001,27.495684,0.000000e+00
25%,42.000000,42.737501,41.500000,41.987501,40.333071,5.317865e+05
50%,43.004999,43.825001,42.410000,43.004999,41.856881,8.460975e+05
75%,45.179998,46.255000,44.472499,45.155000,45.025000,1.372988e+06
max,51.410000,52.169998,50.419998,51.950001,51.950001,7.268611e+06


In [35]:
df_bolsa.dtypes

c_Open         float64
c_High         float64
c_Low          float64
c_Close        float64
c_Adj_Close    float64
c_Volume         int64
d_date          object
dtype: object

In [36]:
df_bolsa.d_date = pd.to_datetime(df_bolsa.d_date)

(c) Verifique que los datos estén ordenados por fecha

In [37]:
df_bolsa.sort_values('d_date', inplace=True)

(d) ¿Cuál es la diferencia promedio entre fechas?

In [38]:
df_bolsa.d_date.diff().describe() # La respuesta es 1 día

count                      5039
mean     1 days 00:18:06.346497
std      0 days 00:08:25.902877
min             1 days 00:04:00
25%             1 days 00:11:00
50%             1 days 00:18:00
75%             1 days 00:26:00
max             1 days 00:32:00
Name: d_date, dtype: object

(e) (TABLA 1) Generé las siguientes variables sin cambiar la frecuencia de los registros (con fechas originales, sin agrupaciones):

Realice el desfase de la variable ”Cierre” de tal forma que los valores por
registro le corresponda el valor de ”Cierre” de dos periodos posteriores
- Variable ”diff max min” que sea la diferencia de los valores High con los valores Low
- Variable ”pct cie ape” cambio porcentual entre el valor de cierre y apertura
- Variable "ratio" que corresponda a los valores de cierre entre los valores de apertura
- Cree las siguientes variables para esta lista: [’Open’, ’High’, ’Low’, ’Volume’]
  - Columnas de diferencia de 1 periodo a 3 per ́ıodos para cada variable
  - Columnas de ventana de tiempo de 1 ,..., 3 para cada variable utilizando la media
  - Columnas de cambio porcentual de 1 periodo a 3 per ́ıodos para cada variable
  - Generar el cuadrado de cada variable

In [39]:
df_bolsa["c_Cierre_tgt"] = df_bolsa.c_Close.shift(2)

df_bolsa['diff_max _min'] = df_bolsa.c_High - df_bolsa.c_Low

df_bolsa["pct_cie_ape"] = (df_bolsa.c_Close - df_bolsa.c_Open)/df_bolsa.c_Open

df_bolsa['ratio'] = df_bolsa.c_Close/df_bolsa.c_Open

In [40]:
l = ['c_Open', 'c_High', 'c_Low', 'c_Volume']

for col in l:
    df_bolsa[col+'_cuadrado'] = df_bolsa[col]**2
    for k in range(3):
        df_bolsa[col+('_diff_%d' %(k+1))] = df_bolsa[col].diff(k+1)
        df_bolsa[col+('_mean_%d' %(k+1))] = df_bolsa[col].rolling(k+1).mean()
        df_bolsa[col+('_pct_%d' %(k+1))] = df_bolsa[col].pct_change(k+1)
    


In [41]:
df_ori = df_bolsa.copy()

In [42]:
df_ori.head(3)

,c_Open,c_High,c_Low,c_Close,c_Adj_Close,c_Volume,d_date,c_Cierre_tgt,diff_max _min,pct_cie_ape,...,c_Volume_cuadrado,c_Volume_diff_1,c_Volume_mean_1,c_Volume_pct_1,c_Volume_diff_2,c_Volume_mean_2,c_Volume_pct_2,c_Volume_diff_3,c_Volume_mean_3,c_Volume_pct_3
0,38.290001,40.160000,38.099998,39.880001,38.219864,3063098,2004-12-01 00:00:00,NaN,2.060002,0.041525,...,9382569357604,NaN,3063098.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,39.880001,40.139999,38.590000,40.000000,38.334866,469910,2004-12-02 00:13:00,NaN,1.549999,0.003009,...,220815408100,-2593188.0,469910.0,-0.846590,NaN,1766504.0,NaN,NaN,NaN,NaN
2,39.799999,41.959999,39.520000,40.790001,39.091980,6223049,2004-12-03 00:38:00,39.880001,2.439999,0.024874,...,38726338856401,5753139.0,6223049.0,12.243066,3159951.0,3346479.5,1.031619,NaN,3252019.0,NaN


(f) (TABLA 2) Cambie la frecuencia de las fechas de tal forma que los registros sean cada 3 días y genere al menos cuatro nuevas variables por cada variable original. No olvide hacer el desfase de la variable objetivo.

In [43]:
df_bolsa2 = df_bolsa[['c_Open','c_High','c_Low','c_Close','c_Adj_Close','c_Volume', 'd_date']].copy()
df_bolsa2.set_index('d_date', inplace=True)

In [44]:
X=df_bolsa2[[x for x in df_bolsa2 if x!="c_Close"]]
y=df_bolsa2[["c_Close"]]

In [45]:
tgt=y.resample("3D").mean()

In [46]:
##AGGREGATION FUNCTIONS
def minPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).min()
def maxPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).max()
def meanPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).mean()
def stdPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).std()
def sumPctChange(x):
    return x.pct_change().replace([np.inf, -np.inf], np.nan).sum()
def minChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).min()
def maxChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).max()
def meanChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).mean()
def stdChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).std()
def sumChange(x):
    return x.diff().replace([np.inf, -np.inf], np.nan).sum()

In [47]:
X_aux=X.resample("3D").agg(["min", "max", "mean", "sum", "std", "median",minPctChange,sumChange])

In [48]:
X_aux.columns = [ name+"|"+func for name, func in X_aux.columns]

In [49]:
df_3_days=pd.concat([X_aux,tgt.shift(-1)],axis=1)

In [51]:
df_3_days.head(3)

,c_Open|min,c_Open|max,c_Open|mean,c_Open|sum,c_Open|std,c_Open|median,c_Open|minPctChange,c_Open|sumChange,c_High|min,c_High|max,...,c_Adj_Close|sumChange,c_Volume|min,c_Volume|max,c_Volume|mean,c_Volume|sum,c_Volume|std,c_Volume|median,c_Volume|minPctChange,c_Volume|sumChange,c_Close
d_date,,,,,,,,,,,,,,,,,,,,,
2004-12-01,38.290001,39.880001,39.323334,117.970001,0.895786,39.799999,-0.002006,1.509998,40.139999,41.959999,...,0.872116,469910,6223049,3.252019e+06,9756057,2.881219e+06,3063098.0,-0.846590,3159951,41.946667
2004-12-04,40.849998,42.020000,41.556666,124.669997,0.621799,41.799999,0.005263,1.170002,42.299999,42.520000,...,0.421680,1170484,3564162,2.215200e+06,6645599,1.225499e+06,1910953.0,-0.463842,-2393678,42.446667
2004-12-07,42.200001,43.360001,42.636667,127.910000,0.630900,42.349998,-0.023293,0.149997,42.840000,43.360001,...,-1.121296,799734,1906038,1.505892e+06,4517675,6.133591e+05,1811903.0,-0.558622,94135,42.630001


## FIN